In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import uniform

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pickle.load(open("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/tfidf_pca_result_final_500.pickle", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_tfidf_pca_result_final_500.pickle")).drop(columns = ['doc_id'])

gc.collect()

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time

model = LogisticRegression(max_iter=10000000)

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=30, cv=3, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print("Model Operation Completed in :",(e-s)/60,"min")

In [0]:
model = LogisticRegression(max_iter=10000000, penalty = 'l2', C = 2.8322903)

import time

s = time.time()
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []

pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

In [0]:
max_cutoff = cutoff[f1 == max(f1)]
max_cutoff

array([0.31])

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = model.predict(X_pred)
probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
1,1869 of 1968. appeal under s. 116-a of the rep...,0.0,0.609307
2,"16 of 1967. s. v. gupte, m. m. tiwari, h. k. p...",0.0,0.633290
5,"rajasthan and k. b. mehta, for the state of ra...",0.0,0.620992
13,"we are concerned only with one of them, namely...",0.0,0.517763
14,"838 article 191 of the constitution, which is ...",0.0,0.632130
...,...,...,...
6923,"but the word ""reduction"" has no such limitatio...",48.0,0.579168
6949,"the learned counsel, however, could not substa...",48.0,0.547999
7046,13. the point now is as to what orders are to ...,48.0,0.509177
7072,"21 'and 41 -of 1952, filed in the high court o...",49.0,0.524486
